In [50]:
from PIL import Image
import numpy as np
import tifffile
import re
import cv2
import matplotlib.pyplot as plt
import os
def parse_value(text, keyword):
    pattern = re.escape(keyword) + r'\s*=\s*"([^"]*)"'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
    
def get_metadata(image_path):
    with tifffile.TiffFile(image_path) as tif:
        metadata = tif.pages[0].tags
        tif_tags = {}
        tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')

    s = tif_tags['XMP']
    # print(s)
    dwarp_data = "drone-dji:DewarpData"
    dwarp_data_para = parse_value(s, dwarp_data)
    dwarp_coefficients  = [float(x) for x in dwarp_data_para[11:].split(',')]
    calibrated_hmatrix = "drone-dji:DewarpHMatrix"
    calibrated_hmatrix = [float(x) for x in parse_value(s, calibrated_hmatrix).split(',')]
    # print(len(calibrated_hmatrix), calibrated_hmatrix)
    calibrated_hmatrix = np.array(calibrated_hmatrix).reshape((3,3))
    # print(calibrated_hmatrix)
    centre_x_para = 'drone-dji:CalibratedOpticalCenterX'
    centre_y_para = 'drone-dji:CalibratedOpticalCenterY'
    vignettingData_para = 'drone-dji:VignettingData'
    center_x = float(parse_value(s, centre_x_para))
    center_y = float(parse_value(s, centre_y_para))
    vignettingData_str = parse_value(s, vignettingData_para)
    vignetting_data = [float(x) for x in vignettingData_str.split(',')]  # Convert string to list of floats
    return center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix

def undistort_image(image_path, outpath_img, coefficients):
    center_x, center_y = coefficients[0], coefficients[1]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = coefficients[2]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = 2200.899902343750, 2200.219970703125, 10.609985351562, -6.575988769531, 0.008104680106, -0.042915198952, -0.000333522010, 0.000239991001, 0.000000000000
    matrix = np.array([[fx, 0, center_x+cx], [0, fy, center_y+cy], [0, 0, 1]])
    dist = np.array([k1, k2, p1, p2, k3])
    # print("md", matrix, dist)
    with Image.open(image_path) as img:
        w, h = img.size
        # img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        # print(img_norm.max(), img_norm.min())
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, dist, (w,h), 1, (w,h))
        dst = cv2.undistort(np.array(img), matrix, dist, newcameramtx)
        # print(np.array_equal(np.array(img), dst))
        # print(dst.max(), dst.min())
        # undistorted_image = Image.fromarray(dst)
    cv2.imwrite(outpath_img, dst)

def apply_vignetting_correction(image_path,outpath_img):
    with Image.open(image_path) as img:
        width, height = img.size
        center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix = get_metadata(image_path)
        # print(center_x, center_y)
        correction_img = np.zeros((height, width))
        for x in range(width):
            for y in range(height):
                r = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
                correction_value = sum([k * (r ** i) for i, k in enumerate((vignetting_data))]) + 1.0
                correction_img[y, x] = correction_value
        img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        corrected_img = img_norm * correction_img
        corrected_img = np.clip(corrected_img, 0, 255).astype(np.uint8)
        # corrected_image = Image.fromarray(corrected_img)
    cv2.imwrite(outpath_img, corrected_img)
    return (center_x, center_y, dwarp_coefficients, calibrated_hmatrix)

def phase_alignment(image_path,outpath_img, parameters):
    calibrated_hmatrix = parameters[-1]
    calibrated_hmatrix = np.array([[9.891065e-01, 1.740813e-02, -1.592078e+01],
                                   [-1.568817e-02, 9.885082e-01, 3.766531e+01],
                                   [1.083204e-06, 5.127963e-07, 1.000000e+00]])
    with Image.open(image_path) as img:
        w, h = img.size
        # print(w, h)
        transformed_image = cv2.warpPerspective(np.array(img), calibrated_hmatrix, (w, h))
        # print(np.array_equal(np.array(img), transformed_image))
        # transformed_image = Image.fromarray(transformed_image)
    cv2.imwrite(outpath_img, transformed_image)

def ecc_alignment(image1_path, image2_path, out_name):
    image1 = cv2.imread(image1_path, cv2.IMREAD_UNCHANGED)
    image2 = cv2.imread(image2_path, cv2.IMREAD_UNCHANGED)
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
    aligned_image2 = cv2.warpPerspective(image2, M, (image1.shape[1], image1.shape[0]))
    print(np.array_equal(aligned_image2, image2))
    cv2.imwrite(out_name, aligned_image2)

# image_path = r"images\DJI_20230928151039_0001_MS_R.TIF"
# outpath_img = r"DJI_20230928151039_0001_MS_R.TIF"
# parameters = apply_vignetting_correction(image_path,outpath_img)
# undistort_image(outpath_img, 'undistort_'+outpath_img, parameters)
# phase_alignment('undistort_'+outpath_img, 'phase_undistort_'+outpath_img, parameters)
# ecc_alignment(r'NIR\phase_undistort_DJI_20230928151039_0001_MS_NIR.TIF', r'images\DJI_20230928151039_0001_D.JPG')

out_dir = r"C:\Users\User\Downloads\MS_Processed"
for im in os.listdir(r"C:\Users\User\Downloads\MS_rawImages"):
    im = os.path.join(r"C:\Users\User\Downloads\MS_rawImages", im)
    imname = os.path.basename(im)
    # if imname.endswith('NIR.TIF'):
    #     print(imname)
    #     img_uint16 = cv2.imread(im, cv2.IMREAD_UNCHANGED)
    #     # print(img_uint16)
    #     img_uint8 = cv2.convertScaleAbs(img_uint16, alpha=(255/65535))

    #     cv2.imwrite(os.path.join(out_dir, imname), img_uint8)
        
        
    if not imname.endswith('NIR.TIF'):
        ref_file = '_'.join(im.split('_')[:-1]) + '_NIR.TIF'
        print(im, ref_file)
    # parameters = apply_vignetting_correction(im, os.path.join(out_dir, imname))
    # undistort_image(os.path.join(out_dir, imname), os.path.join(out_dir, imname), parameters)
    # print(ref_file, os.path.join(out_dir, imname))
        ecc_alignment(ref_file, os.path.join(out_dir, imname), os.path.join(out_dir, imname))
    break



# image_path = r"images\DJI_20230928151039_0001_MS_R.TIF"
# outpath_img = r"DJI_20230928151039_0001_MS_R.TIF"
# parameters = apply_vignetting_correction(image_path,outpath_img)
# undistort_image(outpath_img, 'undistort_'+outpath_img, parameters)
# phase_alignment('undistort_'+outpath_img, 'phase_undistort_'+outpath_img, parameters)
# ecc_alignment(r'NIR\phase_undistort_DJI_20230928151039_0001_MS_NIR.TIF', r'images\DJI_20230928151039_0001_D.JPG')

DJI_20231015133246_0001_MS_G.TIF
['_segments', 'delete', 'delete_all', 'get', 'get_all', 'get_file', 'get_thumbnail', 'has_exif', 'list_all']


UnpackError: 

+--------+------------+-------+-------+------------------------+
| Offset | Access     | Value | Bytes | Format                 |
+--------+------------+-------+-------+------------------------+
|        |            |       |       | TiffHeader (Structure) |
| 0      | byte_order | 32    | 00 20 | tiff_byte_order        |
+--------+------------+-------+-------+------------------------+

ValueError occurred during unpack operation:

32 is not a valid TiffByteOrder

In [37]:
import piexif
from PIL import Image
fname_1=r"C:\Users\User\Downloads\MS_rawImages\DJI_20231015133339_0025_MS_G.TIF"
img = Image.open(fname_1)
exif_dict = piexif.load(fname_1)
print(exif_dict)

{'0th': {254: 0, 256: 2592, 257: 1944, 258: 16, 259: 1, 262: 1, 270: b'default', 271: b'DJI', 272: b'M3M', 273: 9728, 274: 1, 277: 1, 279: 10077696, 305: b'10.12.05.45', 306: b'2023:10:15 13:33:39', 700: (60, 63, 120, 112, 97, 99, 107, 101, 116, 32, 98, 101, 103, 105, 110, 61, 34, 239, 187, 191, 34, 32, 105, 100, 61, 34, 87, 53, 77, 48, 77, 112, 67, 101, 104, 105, 72, 122, 114, 101, 83, 122, 78, 84, 99, 122, 107, 99, 57, 100, 34, 63, 62, 10, 60, 120, 58, 120, 109, 112, 109, 101, 116, 97, 32, 120, 109, 108, 110, 115, 58, 120, 61, 34, 97, 100, 111, 98, 101, 58, 110, 115, 58, 109, 101, 116, 97, 47, 34, 62, 10, 32, 60, 114, 100, 102, 58, 82, 68, 70, 32, 120, 109, 108, 110, 115, 58, 114, 100, 102, 61, 34, 104, 116, 116, 112, 58, 47, 47, 119, 119, 119, 46, 119, 51, 46, 111, 114, 103, 47, 49, 57, 57, 57, 47, 48, 50, 47, 50, 50, 45, 114, 100, 102, 45, 115, 121, 110, 116, 97, 120, 45, 110, 115, 35, 34, 62, 10, 32, 32, 60, 114, 100, 102, 58, 68, 101, 115, 99, 114, 105, 112, 116, 105, 111, 110, 3

In [43]:
!pip install exif

  Obtaining dependency information for plum-py<2.0.0,>=0.5.0 from https://files.pythonhosted.org/packages/2f/da/8a9bdf5d3ea4b13d3142e0c77d6e85e4c2668aef6e4b9d5e9a6619049bdc/plum_py-0.8.7-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/70.0 kB ? eta -:--:--
   ----------------------- ---------------- 41.0/70.0 kB ? eta -:--:--
   ----------------------------------- ---- 61.4/70.0 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 70.0/70.0 kB 642.5 kB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\aniso8601-9.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\ansi2html-1.9.0rc1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\flask_restful-0.3.10-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\nvgpu-0.10.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\pynvml-11.5.0-py3.11.egg is deprecated. p

In [13]:
!pip install tifffile

DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\aniso8601-9.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\ansi2html-1.9.0rc1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\flask_restful-0.3.10-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\nvgpu-0.10.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\users\user\anaconda3\lib\site-packages\pynvml-11.5.0-py3.11.egg is deprecated. p

In [5]:
# import tifffile
# with tifffile.TiffFile(r"C:\Users\User\Desktop\Fizza\MultiSpectral-Image-Correction\outputset\DJI_20231015133246_0001_MS_G.TIF") as tif:
#     metadata = tif.pages[0].tags
#     tif_tags = {}
#     tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')
#     s = open("meta2.xml", 'w', encoding='utf-8')
#     s.write(tif_tags['XMP'])
import xml.etree.ElementTree as ET

# Load the XML file
tree = ET.parse('meta.xml')
root = tree.getroot()

# Define the namespaces to keep (in this case, Camera and xmp)
namespaces_to_keep = ['Camera']

# Loop through all elements in the XML
for elem in root:
    # Remove namespaces that are not in the list to keep
    for key in list(elem.attrib.keys()):
        namespace = key.split('}', 1)[0] + '}'
        if namespace not in namespaces_to_keep:
            del elem.attrib[key]

# Save the modified XML
tree.write('output.xml')


In [ ]:
from PIL import Image
import numpy as np
import tifffile
import re
import cv2
import matplotlib.pyplot as plt
import os
from itertools import groupby
def parse_value(text, keyword):
    pattern = re.escape(keyword) + r'\s*=\s*"([^"]*)"'
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None
def get_metadata(image_path):
    with tifffile.TiffFile(image_path) as tif:
        metadata = tif.pages[0].tags
        tif_tags = {}
        tif_tags['XMP'] = tif.pages[0].tags[700].value.decode('utf-8')
    s = tif_tags['XMP']
    # print(s)
    dwarp_data = "drone-dji:DewarpData"
    dwarp_data_para = parse_value(s, dwarp_data)
    dwarp_coefficients  = [float(x) for x in dwarp_data_para[11:].split(',')]
    calibrated_hmatrix = "drone-dji:DewarpHMatrix"
    calibrated_hmatrix = [float(x) for x in parse_value(s, calibrated_hmatrix).split(',')]
    # print(len(calibrated_hmatrix), calibrated_hmatrix)
    calibrated_hmatrix = np.array(calibrated_hmatrix).reshape((3,3))
    # print(calibrated_hmatrix)
    centre_x_para = 'drone-dji:CalibratedOpticalCenterX'
    centre_y_para = 'drone-dji:CalibratedOpticalCenterY'
    vignettingData_para = 'drone-dji:VignettingData'
    center_x = float(parse_value(s, centre_x_para))
    center_y = float(parse_value(s, centre_y_para))
    vignettingData_str = parse_value(s, vignettingData_para)
    vignetting_data = [float(x) for x in vignettingData_str.split(',')]  # Convert string to list of floats
    return center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix
def undistort_image(image_path, outpath_img, coefficients):
    center_x, center_y = coefficients[0], coefficients[1]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = coefficients[2]
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = 2200.899902343750, 2200.219970703125, 10.609985351562, -6.575988769531, 0.008104680106, -0.042915198952, -0.000333522010, 0.000239991001, 0.000000000000
    matrix = np.array([[fx, 0, center_x+cx], [0, fy, center_y+cy], [0, 0, 1]])
    dist = np.array([k1, k2, p1, p2, k3])
    # print("md", matrix, dist)
    with Image.open(image_path) as img:
        w, h = img.size
        # img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        # print(img_norm.max(), img_norm.min())
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, dist, (w,h), 1, (w,h))
        dst = cv2.undistort(np.array(img), matrix, dist, newcameramtx)
        # print(np.array_equal(np.array(img), dst))
        # print(dst.max(), dst.min())
        # undistorted_image = Image.fromarray(dst)
    cv2.imwrite(outpath_img, dst)
def apply_vignetting_correction(image_path,outpath_img):
    with Image.open(image_path) as img:
        width, height = img.size
        center_x, center_y, vignetting_data, dwarp_coefficients, calibrated_hmatrix = get_metadata(image_path)
        # print(center_x, center_y)
        correction_img = np.zeros((height, width))
        for x in range(width):
            for y in range(height):
                r = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
                correction_value = sum([k * (r ** i) for i, k in enumerate((vignetting_data))]) + 1.0
                correction_img[y, x] = correction_value
        img_norm = cv2.normalize( np.array(img), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        corrected_img = img_norm * correction_img
        corrected_img = np.clip(corrected_img, 0, 255).astype(np.uint8)
        # corrected_image = Image.fromarray(corrected_img)
    cv2.imwrite(outpath_img, corrected_img)
    return (center_x, center_y, dwarp_coefficients, calibrated_hmatrix)
def phase_alignment(image_path,outpath_img, parameters):
    calibrated_hmatrix = parameters[-1]
    calibrated_hmatrix = np.array([[9.891065e-01, 1.740813e-02, -1.592078e+01],
                                   [-1.568817e-02, 9.885082e-01, 3.766531e+01],
                                   [1.083204e-06, 5.127963e-07, 1.000000e+00]])
    with Image.open(image_path) as img:
        w, h = img.size
        # print(w, h)
        transformed_image = cv2.warpPerspective(np.array(img), calibrated_hmatrix, (w, h))
        # print(np.array_equal(np.array(img), transformed_image))
        # transformed_image = Image.fromarray(transformed_image)
    cv2.imwrite(outpath_img, transformed_image)
def ecc_alignment(image1_path, image2_path, out_name):
    image1 = cv2.imread(image1_path, cv2.IMREAD_UNCHANGED)
    image2 = cv2.imread(image2_path, cv2.IMREAD_UNCHANGED)
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(image1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
    matched_image = cv2.drawMatches(image1, keypoints1, image2, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
    aligned_image2 = cv2.warpPerspective(image2, M, (image1.shape[1], image1.shape[0]))
    print(np.array_equal(aligned_image2, image2))
    cv2.imwrite(out_name, aligned_image2)
def find_files(folder, types):
    return [entry.path for entry in os.scandir(folder) if(entry.is_file() and os.path.splitext(entry.name)[1].lower() in types)]
# Sorting key function
def sort_key(item):
    # Define the order of the band types
    band_order = {'G': 0, 'NIR': 1, 'R': 2, 'RE': 3}
    image_number_str = item.split('DJI_')[-1].split('_')[1]
    image_number = int(image_number_str)
    band_type = item.split('')[-1].split('.')[0].split('')[-1]  # Extracting band type
    return image_number, band_order.get(band_type, float('inf'))  # Handling unexpected band types
def pre_process_M3M(input_img,out_img):
    parameters = apply_vignetting_correction(input_img,out_img)
    undistort_image(out_img, out_img, parameters)
    phase_alignment(out_img, out_img, parameters)
    ecc_alignment(r'NIR\phase_undistort_DJI_20230928151039_0001_MS_NIR.TIF', r'images\DJI_20230928151039_0001_D.JPG')

def main(input_folder,output_folder):
    ms_photos = find_files(input_folder, [".tif"])
    rgb_photos = find_files(input_folder, [".jpg"])
    # Sort the list
    sorted_data = sorted(ms_photos, key=sort_key)
    # Group by image number
    grouped = [list(g) for _, g in groupby(sorted_data, key=lambda x: x.split('DJI_')[-1].split('_')[1])]
    for x in grouped:
        pre_process_M3M(x[0],os.path.join(output_folder,'pre_processed_' + x[0].split('\\')[-1]))
        
if _name_ == '_main_':
    input_folder = r"C:\Users\User\Desktop\imagery alignment\input_Data"
    output_folder = r"C:\Users\User\Desktop\imagery alignment\output_Data"
    main(input_folder,output_folder)

In [9]:
import cv2
folder = r"E:\MS\Plant_wise\test"

from PIL import Image

def rotate_tif(input_path, output_path, angle):
    with Image.open(input_path) as img:
        rotated_img = img.rotate(angle, expand=True)
        rotated_img.info = img.info
        rotated_img.save(output_path)

for i in os.listdir(folder):
    if i.endswith('tif'):
        print(i)
        input_path = os.path.join(folder, i)
        output_path = os.path.join(folder, 'rotated_' + i)
        angle = 90
        rotate_tif(input_path, output_path, angle)

farm_0_1.tif
farm_0_2.tif
farm_0_3.tif
farm_0_4.tif
farm_0_5.tif


In [1]:
import cv2

# Load your RGB and multispectral images
rgb_image = cv2.imread('images\DJI_20230928151039_0001_D.JPG')
multispectral_image = cv2.imread('images\DJI_20230928151039_0001_MS_G.TIF')

# Convert both images to grayscale (optional but recommended)
rgb_gray = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
multispectral_gray = cv2.cvtColor(multispectral_image, cv2.COLOR_BGR2GRAY)

# Detect keypoints and descriptors (using ORB, you can choose a different method)
orb = cv2.ORB_create()
keypoints1, descriptors1 = orb.detectAndCompute(rgb_gray, None)
keypoints2, descriptors2 = orb.detectAndCompute(multispectral_gray, None)

# Match keypoints
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)
matches = sorted(matches, key=lambda x: x.distance)

# Extract corresponding points
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

# Find the transformation matrix
M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# Apply the transformation to align the RGB image with the multispectral image
aligned_rgb = cv2.warpPerspective(rgb_image, M, (multispectral_image.shape[1], multispectral_image.shape[0]))

# Save the aligned RGB image
cv2.imwrite('aligned_rgb_image.jpg', aligned_rgb)


KeyboardInterrupt: 